In [3]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import *
from tinyshift.stats import trailing_window, mad
from tinyshift.series import trend_significance
from numpy.random import standard_normal

In [4]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [5]:
r = standardize_returns(data["Close"]["PETR4.SA"], log=False)

# RSI

In [6]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [7]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [8]:
vol_ent = trailing_window(r, rolling_window=60, func=sample_entropy, m=1)

In [9]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [10]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_rsi,
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [11]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"]))

np.int64(56)

In [12]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"], center=np.median, spread=mad))

np.int64(175)

# Hampel Filter

In [13]:
hf = hampel_filter(data["Close"]["PETR4.SA"], rolling_window=7)

In [14]:
data["Close"]["PETR4.SA"][hf]

Date
2024-05-22    30.161886
2024-07-04    31.138041
2024-10-09    31.938402
2025-02-07    33.884270
Name: PETR4.SA, dtype: float64

# ADI-CV Framework

In [15]:
adi_cv(r)

(np.float64(1.0), np.float64(1.9886348019242868e+34))

# Sampen

In [46]:
sample_entropy(data["Close"]["PETR4.SA"], m=3, detrend=True)

np.float64(0.5921725376889516)

In [39]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Permutation Entropy

In [42]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=True)

np.float64(0.9635729769639065)

In [20]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=False)

np.float64(2.4908000121599487)

# Stability Index

In [49]:
stability_index(data["Close"]["PETR4.SA"], detrend=True)

np.float64(0.5333202409007594)

In [52]:
stability_index(pure_sine)

np.float64(0.8656234267494547)

# Theorical Limit

In [58]:
theoretical_limit(data["Close"]["PETR4.SA"], m=3)

np.float64(0.036427023036093464)

In [59]:
theoretical_limit(pure_sine, m=4, delay=1)

np.float64(0.7114310780157527)

# Foreca

In [62]:
foreca(data["Close"]["PETR4.SA"])

0.6003496948076306

In [63]:
trailing_window(data["Close"]["PETR4.SA"], rolling_window=60, func=foreca)

array([0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.65926832, 0.65926832, 0.65926832, 0.65926832, 0.65926832,
       0.67173143, 0.66908263, 0.66265217, 0.64846261, 0.65486014,
       0.63313947, 0.61692486, 0.56839568, 0.5188092 , 0.48243043,
       0.50070087, 0.51322694, 0.50129878, 0.49858305, 0.49071

In [64]:
foreca(pure_sine)

0.9993976808641186

In [28]:
trailing_window(pure_sine, rolling_window=60, func=foreca)

array([0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.85659439, 0.83599743, 0.81701635, 0.79995579, 0.78488091,
       0.77171463, 0.76030651, 0.75047711, 0.74204419, 0.73483677,
       0.72870169, 0.72350584, 0.7191359 , 0.71549702, 0.71251

# Hurst's Exponent

In [70]:
hurst_exponent(r, d=0)

(0.6285298895961746, 0.031450516714001515)

In [71]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [72]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [73]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [74]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [75]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)